In [ ]:
#libraries
import pandas as pd
import numpy as np
import datetime as dt
pd.set_option("display.max_rows", None, "mode.chained_assignment", None)
import numpy as np
import matplotlib.pyplot as plt
import json
from json import loads, dumps
import datetime as dt
import plotly
import plotly.express as px

In [ ]:
# read csv
care_site = pd.read_csv('../datasets/CARE_SITE.csv')
cohort_filter = pd.read_csv('../datasets/COHORT_FILTER.csv')
concept_class= pd.read_csv("../datasets/CONCEPT_CLASS.csv")
concept = pd.read_csv("../datasets/CONCEPT.csv")
condition_occurrence = pd.read_csv("../datasets/CONDITION_OCCURRENCE.csv")
domain = pd.read_csv('../datasets/DOMAIN.csv')
drug_exposure = pd.read_csv('../datasets/DRUG_EXPOSURE.csv')
location = pd.read_csv('../datasets/LOCATION.csv')
measurement = pd.read_csv('../datasets/MEASUREMENT.csv', low_memory=False)
observation = pd.read_csv('../datasets/OBSERVATION.csv')
person = pd.read_csv('../datasets/PERSON.csv')
procedure_occurrence = pd.read_csv('../datasets/PROCEDURE_OCCURRENCE.csv')
provider = pd.read_csv('../datasets/PROVIDER.csv')
vocabulary = pd.read_csv('../datasets/VOCABULARY.csv')

In [ ]:
#concept lookup in concept df
concept_lookup = {c.concept_id: c.concept_name for c in concept.itertuples()}

In [ ]:
#function to make labels
def make_labels(df):
    for c in df.columns:
        if 'concept_id' in c:
            df[c.replace('_id', '_label')] = df[c].map(concept_lookup)
        if 'concept_id' in c or 'source' in c or len(df[df[c].notna()])==0:
            df = df.drop(c, axis=1)
    return df

In [ ]:
#
care_site_labelled = make_labels(care_site)
cohort_filter_labelled = make_labels(cohort_filter)
condition_occurrence_labelled = make_labels(condition_occurrence)
domain_labelled = make_labels(domain)
drug_exposure_labelled = make_labels(drug_exposure)
location_labelled = make_labels(location)
measurement_labelled = make_labels(measurement)
observation_labelled = make_labels(observation)
person_labelled = make_labels(person)
procedure_occurrence_labelled = make_labels(procedure_occurrence)
provider_labelled = make_labels(provider)
vocabulary_labelled = make_labels(vocabulary)

In [ ]:
person_code = 

In [ ]:
meas_indiv = measurement_labelled.loc[measurement_labelled['person_id']==person_code]

In [ ]:
### measurements
meas_weight = meas_indiv.loc[meas_indiv['measurement_concept_label'].isin(['Body weight','Weight change'])]
#Merge weight measurements with weight differences from last measurement
meas_weight['value_as_number']=meas_weight['value_as_number'].apply(str)
meas_grouped = meas_weight.groupby(['measurement_date'])['value_as_number'].transform(lambda x: ' kg,<br>Weight change: '.join(x))
meas_weight['value_as_number'] = meas_grouped
meas_weight['content'] = 'Body weight: ' + meas_weight['value_as_number'] + ' kg'
#Rename columns
meas_weight.rename(columns={'measurement_date':'start'}, inplace=True)
meas_weight['group'] = 'measurement'
meas_weight['className'] = 'measurements'
meas_weight.drop_duplicates(subset=['content', 'start'],keep='last',inplace=True)

In [ ]:
meas_indiv

In [ ]:
meas_graphs = meas_indiv.loc[meas_indiv['measurement_concept_label'].isin(['Body surface area', 'Body height', 'Body weight', 'Weight change'])]
meas_graphs

In [ ]:
meas_graphs['measurement_concept_label']=meas_graphs['measurement_concept_label'].replace({
    'Body weight':'Body weight (kg)', 
    'Weight change':'Weight change (%)',
    'Body height': 'Body height (cm)',
    'Body surface area': 'Body surface area (m\u00B2)'})

In [ ]:
weight_fig = px.scatter(meas_graphs, x="measurement_date", y="value_as_number", 
facet_col="measurement_concept_label", facet_col_wrap=2,
facet_row_spacing=0.1, facet_col_spacing=0.05)
weight_fig.update_yaxes(matches=None, showticklabels=True, title="")
weight_fig.update_xaxes(title="")
weight_fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
weight_fig.update_layout(margin=dict(l=0, r=10, t=20, b=10))
weight_fig.show()
weight_fig.write_html("weight-plot.html", full_html=False, include_plotlyjs=False)